### End-to-end pipeline to split a video

In [1]:
from src.bouldering.media.video.video import Video

In [2]:
# 1. Read the input video
VIDEO_PATH = "../data/video.mp4"

video = Video.read(VIDEO_PATH)
print(video.metadata)

{'video': {'fps': 25.0, 'n_frames': 10500, 'resolution': (1280, 720)}, 'audio': {'sample_rate': 44100, 'duration': 420.0025396825397}}


In [3]:
# Split the video
# macro splitter: OCR based
# micro splitter: Content based

from src.bouldering.models.ocr.splitter import SceneSplitterOCR
from src.bouldering.structure.scenes.ocr import OCRSplitter
from src.bouldering.structure.scenes.content import ContentSplitter
from src.bouldering.structure.pipeline import ScenePipeline

# OCR splitter
ocr_logic = SceneSplitterOCR(
    langs=["en"],
    crop_box=[0, 0.5, 0.5, 1],  # <-- relative coords
    fx=0.5,
    fy=0.5,
    stride=3,
    batch_size=16,
    smooth_window=5,
    majority_ratio=0.6,
    require_number=True,
)

ocr_splitter = OCRSplitter(ocr_logic)

# Content splitter
content_splitter = ContentSplitter(
    threshold=27.0,
    min_scene_len_sec=1.5,
    downscale=2,
)

# Wrapper pipeline
pipeline = ScenePipeline(
    macro_splitter=ocr_splitter,
    micro_splitter=content_splitter,
)

/home/souheil_laptop/anaconda3/envs/altaroad310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using CPU. Note: This module is much faster with a GPU.


In [4]:
# run pipeline
scenes = pipeline.run(video.cut(0.0, 40.0))

len(scenes)

OCR scene splitting:   0%|          | 0/1000 [00:00<?, ?frame/s]/home/souheil_laptop/anaconda3/envs/altaroad310/lib/python3.10/site-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)
OCR scene splitting: 100%|██████████| 1000/1000 [01:32<00:00, 10.83frame/s]
Downscale factor will be ignored because auto_downscale=True!


2

In [5]:
scenes

[Scene(start_time=20.4, end_time=31.84, label='boulder_1', metadata={'parent': Scene(start_time=20.4, end_time=40.0, label='boulder_1', metadata={'source': 'ocr'}), 'source': 'ocr+content'}),
 Scene(start_time=31.84, end_time=40.0, label='boulder_1', metadata={'parent': Scene(start_time=20.4, end_time=40.0, label='boulder_1', metadata={'source': 'ocr'}), 'source': 'ocr+content'})]